Preprocessing complet

In [47]:
import pandas as pd
import numpy as np

def preprocess_all(path_train_input: str, path_test_input: str, path_train_output: str, n_train: int = 383610):
    """
    Prétraitement simplifié sans normalisation, uniquement typage et nettoyage.
    """
    print("📥 Chargement des fichiers...")
    test_input_train = pd.read_csv(path_train_input)
    test_input_real = pd.read_csv(path_test_input)
    train_output = pd.read_csv(path_train_output)

    print("🔗 Fusion des inputs...")
    train_input = pd.concat([test_input_train, test_input_real], ignore_index=True)

    print("🧬 Fusion des données sur 'ID' avec train_output...")
    df = train_input.merge(train_output, on="ID", how="left")

    if 'ANNEE_ASSURANCE_x' in df.columns and 'ANNEE_ASSURANCE_y' in df.columns:
        df.drop(columns=['ANNEE_ASSURANCE_y'], inplace=True)
        df.rename(columns={'ANNEE_ASSURANCE_x': 'ANNEE_ASSURANCE'}, inplace=True)

    if "ZONE" in df.columns:
        df["ZONE"] = df["ZONE"].astype(str).str.strip().astype("category")

    print("🎯 Séparation features / cibles...")
    y = df[["FREQ", "CM"]].copy()
    columns_to_drop = ['FREQ', 'CM', 'CHARGE']
    X = df.drop(columns=columns_to_drop, errors='ignore').copy()

    print("🔄 Conversion des colonnes...")
    for col in X.columns:
        try:
            X[col] = pd.to_numeric(X[col], errors='raise')
        except:
            X[col] = X[col].astype(str).str.strip().astype('category')

    print("✂️ Découpage train/test...")
    X_train = X.iloc[:n_train].copy()
    X_test = X.iloc[n_train:].copy()
    y_train_freq = y["FREQ"].iloc[:n_train].copy()
    y_train_cm = y["CM"].iloc[:n_train].copy()

    print("✅ Preprocessing terminé.")
    print(f" - X final : {X.shape}")
    print(f" - y final : {y.shape}")
    print(f" - X_train : {X_train.shape}")
    print(f" - X_test  : {X_test.shape}")
    print(f" - y_train_freq : {y_train_freq.shape}")
    print(f" - y_train_cm   : {y_train_cm.shape}")

    return X_train, X_test, y_train_freq, y_train_cm, df, X, y

X_train, X_test, y_train_freq, y_train_cm, df, X, y = preprocess_all(
    "train_input.csv",
    "test_input.csv",
    "train_output.csv"
)

print("\n📂 Colonnes finales utilisées (X.columns) :")
print(list(X.columns))

📥 Chargement des fichiers...


C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_15640\1676540384.py:9: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  test_input_train = pd.read_csv(path_train_input)
C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_15640\1676540384.py:10: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  test_input_real = pd.read_csv(path_test_input)


🔗 Fusion des inputs...
🧬 Fusion des données sur 'ID' avec train_output...
🎯 Séparation features / cibles...
🔄 Conversion des colonnes...
✂️ Découpage train/test...
✅ Preprocessing terminé.
 - X final : (479462, 374)
 - y final : (479462, 2)
 - X_train : (383610, 374)
 - X_test  : (95852, 374)
 - y_train_freq : (383610,)
 - y_train_cm   : (383610,)

📂 Colonnes finales utilisées (X.columns) :
['ID', 'ACTIVIT2', 'VOCATION', 'TYPERS', 'ANCIENNETE', 'ADOSS', 'CARACT1', 'CARACT2', 'CARACT3', 'INDEM1', 'DUREE_REQANEUF', 'CARACT4', 'CARACT5', 'TYPBAT1', 'INDEM2', 'TYPBAT2', 'FRCH1', 'FRCH2', 'DEROG1', 'DEROG2', 'DEROG3', 'DEROG4', 'DEROG5', 'DEROG6', 'DEROG7', 'DEROG8', 'DEROG9', 'DEROG10', 'DEROG11', 'DEROG12', 'DEROG13', 'DEROG14', 'DEROG15', 'DEROG16', 'TAILLE1', 'TAILLE2', 'CA1', 'CA2', 'CA3', 'KAPITAL1', 'KAPITAL2', 'KAPITAL3', 'KAPITAL4', 'KAPITAL5', 'KAPITAL6', 'KAPITAL7', 'KAPITAL8', 'KAPITAL9', 'KAPITAL10', 'KAPITAL11', 'KAPITAL12', 'KAPITAL13', 'KAPITAL14', 'KAPITAL15', 'KAPITAL16', 

Entrainement des 2 modèles

In [48]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import warnings
import numpy as np

warnings.filterwarnings("ignore", category=ConvergenceWarning)

@ignore_warnings(category=ConvergenceWarning)
def train_model_freq(X_train, y_train_freq, X_test):
    print("🎯 Entraînement modèle FREQ avec Optuna")

    # Échantillonnage
    sample_idx = np.random.choice(X_train.index, size=50000, replace=False)
    X_sample = X_train.loc[sample_idx]
    y_sample = y_train_freq.loc[sample_idx]

    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 150),
            'max_depth': trial.suggest_int('max_depth', 2, 4),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
            'random_state': 42,
            'enable_categorical': True,
            'objective': 'reg:squarederror',
            'n_jobs': -1
        }
        model = XGBRegressor(**params)
        return cross_val_score(model, X_sample, y_sample, cv=2, scoring='r2', n_jobs=-1).mean()

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10, show_progress_bar=True)

    print(f"✅ Best params FREQ : {study.best_params}")
    model = XGBRegressor(**study.best_params, enable_categorical=True, objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train_freq)

    # Sauvegarde
    with open("model_freq.pkl", "wb") as f:
        pickle.dump(model, f)

    y_pred = model.predict(X_test)
    return y_pred, model

################################################################################################

@ignore_warnings(category=ConvergenceWarning)
def train_model_cm(X_train, y_train_cm, X_test):
    print("🎯 Entraînement modèle CM avec Optuna")

    # Échantillonnage
    sample_size = min(50000, len(X_train))
    sample_idx = np.random.choice(X_train.index, size=sample_size, replace=False)
    X_sample = X_train.loc[sample_idx]
    y_sample = y_train_cm.loc[sample_idx]

    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 150),
            'max_depth': trial.suggest_int('max_depth', 2, 5),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
            'enable_categorical': True,
            'objective': 'reg:squarederror',
            'random_state': 42,
            'n_jobs': -1
        }
        model = XGBRegressor(**params)
        return cross_val_score(model, X_sample, y_sample, cv=2, scoring='r2', n_jobs=-1).mean()

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10, show_progress_bar=True)

    print(f"✅ Best params CM : {study.best_params}")
    model = XGBRegressor(**study.best_params, enable_categorical=True, objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train_cm)

    # Sauvegarde
    with open("model_cm.pkl", "wb") as f:
        pickle.dump(model, f)

    y_pred = model.predict(X_test)
    return y_pred, model

y_pred_freq, model_freq = train_model_freq(X_train, y_train_freq, X_test)
y_pred_cm, model_cm = train_model_cm(X_train, y_train_cm, X_test)


[I 2025-04-30 19:23:39,914] A new study created in memory with name: no-name-c1228c48-bef2-4de4-9147-4b25c25e98c4


🎯 Entraînement modèle FREQ avec Optuna


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-04-30 19:23:45,867] Trial 0 finished with value: -0.05803472418450173 and parameters: {'n_estimators': 83, 'max_depth': 3, 'learning_rate': 0.08232072115459067, 'subsample': 0.8115924296753027, 'colsample_bytree': 0.97925126702631, 'reg_lambda': 4.698061053127093, 'reg_alpha': 0.9817493292084134}. Best is trial 0 with value: -0.05803472418450173.
[I 2025-04-30 19:23:49,792] Trial 1 finished with value: -0.006451812457332284 and parameters: {'n_estimators': 58, 'max_depth': 2, 'learning_rate': 0.037891782228211164, 'subsample': 0.7724900397063406, 'colsample_bytree': 0.7436391134922085, 'reg_lambda': 4.895676657464107, 'reg_alpha': 1.1038465624997662}. Best is trial 1 with value: -0.006451812457332284.
[I 2025-04-30 19:23:53,765] Trial 2 finished with value: -0.0035343167298584888 and parameters: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.026393439474402203, 'subsample': 0.7318781305082679, 'colsample_bytree': 0.7422988409285447, 'reg_lambda': 3.5520049882597124, 'r

[I 2025-04-30 19:25:13,221] A new study created in memory with name: no-name-4a03d5b1-0233-40e8-babf-3cc7972e443d


🎯 Entraînement modèle CM avec Optuna


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-04-30 19:25:17,161] Trial 0 finished with value: -0.07532089747462167 and parameters: {'n_estimators': 51, 'max_depth': 5, 'learning_rate': 0.06040529183485253, 'subsample': 0.9327965651059811, 'colsample_bytree': 0.9560267395591965, 'reg_lambda': 3.1752845633728457, 'reg_alpha': 3.616867522552859}. Best is trial 0 with value: -0.07532089747462167.
[I 2025-04-30 19:25:24,119] Trial 1 finished with value: -0.25089688261844767 and parameters: {'n_estimators': 61, 'max_depth': 4, 'learning_rate': 0.09871568222045252, 'subsample': 0.9732896497718513, 'colsample_bytree': 0.9611667627782907, 'reg_lambda': 0.6522146805991352, 'reg_alpha': 1.2997466003963516}. Best is trial 0 with value: -0.07532089747462167.
[I 2025-04-30 19:25:35,746] Trial 2 finished with value: -0.042154051083769684 and parameters: {'n_estimators': 143, 'max_depth': 2, 'learning_rate': 0.040261361753952604, 'subsample': 0.8747242084742213, 'colsample_bytree': 0.8586726218025633, 'reg_lambda': 1.3097268635254733, 'r

Résultat adapté à un fichier

In [51]:
import pandas as pd
import pickle

def preprocess_for_prediction(input_path: str):
    """
    Prétraitement léger pour prédiction (sans normalisation, avec typage).
    """
    print("📥 Chargement du fichier à prédire...")
    X_new = pd.read_csv(input_path)

    if 'ANNEE_ASSURANCE_x' in X_new.columns and 'ANNEE_ASSURANCE_y' in X_new.columns:
        X_new.drop(columns=['ANNEE_ASSURANCE_y'], inplace=True)
        X_new.rename(columns={'ANNEE_ASSURANCE_x': 'ANNEE_ASSURANCE'}, inplace=True)

    print("🔄 Typage des colonnes...")
    for col in X_new.columns:
        try:
            X_new[col] = pd.to_numeric(X_new[col], errors='raise')
        except:
            X_new[col] = X_new[col].astype(str).str.strip().astype('category')
    
    if "ZONE" in X_new.columns:
        X_new["ZONE"] = X_new["ZONE"].astype(str).str.strip().astype("category")

    print(f"✅ Données prêtes : {X_new.shape}")
    return X_new

def generate_submission(input_path: str, output_path: str = "submission.csv"):
    """
    Fonction complète : chargement des modèles, prédiction sur données,
    calcul de CHARGE, et sauvegarde en CSV.
    """
    print("📦 Chargement des modèles...")
    with open("model_freq.pkl", "rb") as f:
        model_freq = pickle.load(f)
    with open("model_cm.pkl", "rb") as f:
        model_cm = pickle.load(f)

    print("🔍 Prétraitement des données...")
    X_to_predict = preprocess_for_prediction(input_path)

    print("📈 Prédictions en cours...")
    y_pred_freq = model_freq.predict(X_to_predict)
    y_pred_cm = model_cm.predict(X_to_predict)

    print("🧮 Construction du DataFrame de sortie...")
    df_submission = pd.DataFrame({
        "ID": X_to_predict["ID"],
        "FREQ": y_pred_freq,
        "CM": y_pred_cm,
        "ANNEE_ASSURANCE": X_to_predict["ANNEE_ASSURANCE"]
    })
    df_submission["CHARGE"] = df_submission["FREQ"] * df_submission["CM"] * df_submission["ANNEE_ASSURANCE"]

    print(f"💾 Sauvegarde dans '{output_path}'...")
    df_submission.to_csv(output_path, index=False)
    print("✅ Fichier généré avec succès.")

generate_submission("test_input.csv")


📦 Chargement des modèles...
🔍 Prétraitement des données...
📥 Chargement du fichier à prédire...


C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_15640\2421288777.py:9: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X_new = pd.read_csv(input_path)


🔄 Typage des colonnes...
✅ Données prêtes : (95852, 374)
📈 Prédictions en cours...
🧮 Construction du DataFrame de sortie...
💾 Sauvegarde dans 'submission.csv'...
✅ Fichier généré avec succès.


In [52]:
import pandas as pd

# Charge le fichier complet
df = pd.read_csv("test_input.csv")

# Prend seulement les 1000 premières lignes
df_small = df.head(1000)

# Sauvegarde dans un nouveau fichier
df_small.to_csv("test_input_for_API.csv", index=False)

print("✅ Nouveau fichier 'test_input_for_API.csv' créé avec 1000 lignes.")


C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_15640\1810994165.py:4: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("test_input.csv")


✅ Nouveau fichier 'test_input_for_API.csv' créé avec 1000 lignes.
